# Higher order functions

## Curried functions

In [10]:
max 4 5

5

In [5]:
(max 4) 5

Line 1: Redundant bracket
Found:
(max 4) 5
Why not:
max 4 5

5

In [6]:
:t max

max :: forall a. Ord a => a -> a -> a

In [12]:
multThree :: (Num a) => a -> a -> a -> a  
multThree x y z = x * y * z  

In [13]:
let multTwoWithNine = multThree 9  
multTwoWithNine 2 3  

54

In [15]:
let multWithEighteen = multTwoWithNine 2
multWithEighteen 10 

180

In [16]:
-- function that compares a number with 100
compareWithHundred :: (Num a, Ord a) => a -> Ordering  
compareWithHundred x = compare 100 x  

In [17]:
-- same as before, but simpler
compareWithHundred :: (Num a, Ord a) => a -> Ordering  
compareWithHundred = compare 100  

In [18]:
-- functions which divides by ten using infix notation
divideByTen :: (Floating a) => a -> a  
divideByTen = (/10)  

In [19]:
-- checks if a character supplied to the function is an uppercase letter
isUpperAlphanum :: Char -> Bool  
isUpperAlphanum = (`elem` ['A'..'Z'])  

## Some higher-orderism is in order

In [20]:
-- function taking a function and applies twice to something
applyTwice :: (a -> a) -> a -> a  
applyTwice f x = f (f x)  

In [21]:
applyTwice (+3) 10 

16

In [22]:
applyTwice (++ " HAHA") "HEY"

"HEY HAHA HAHA"

In [23]:
applyTwice ("HAHA " ++) "HEY" 

"HAHA HAHA HEY"

In [24]:
applyTwice (multThree 2 2) 9

144

In [25]:
applyTwice (3:) [1] 

[3,3,1]

In [26]:
-- takes a function and two lists as parameters and then joins the two lists by applying
-- the function between corresponding elements
zipWith' :: (a -> b -> c) -> [a] -> [b] -> [c]  
zipWith' _ [] _ = []  
zipWith' _ _ [] = []  
zipWith' f (x:xs) (y:ys) = f x y : zipWith' f xs ys  

In [27]:
zipWith' (+) [4,2,5,6] [2,6,2,3]

[6,8,7,9]

In [28]:
zipWith' max [6,3,2,1] [7,3,1,5]

[7,3,2,5]

In [29]:
zipWith' (++) ["foo ", "bar ", "baz "] ["fighters", "hoppers", "aldrin"] 

["foo fighters","bar hoppers","baz aldrin"]

In [30]:
zipWith' (zipWith' (*)) [[1,2,3],[3,5,6],[2,3,4]] [[3,2,2],[3,4,5],[5,4,3]]

[[3,4,6],[9,20,30],[10,12,12]]

In [31]:
-- takes a function and returns a function like the original, but with the first two arguments flipped
flip' :: (a -> b -> c) -> (b -> a -> c)  
flip' f = g  
    where g x y = f y x  

In [32]:
flip' zip [1,2,3,4,5] "hello"

[('h',1),('e',2),('l',3),('l',4),('o',5)]

## Maps and Filters

In [36]:
-- takes a function and a list and applies that function to every element in the list, producing a new list
map :: (a -> b) -> [a] -> [b]  
map _ [] = []  
map f (x:xs) = f x : map f xs  

In [35]:
map (+3) [1,5,3,1,6]  

[4,8,6,4,9]

In [37]:
-- takes a predicate and a list and then returns the list of elements that satisfy the predicate
filter :: (a -> Bool) -> [a] -> [a]  
filter _ [] = []  
filter p (x:xs)   
    | p x       = x : filter p xs  
    | otherwise = filter p xs 

In [38]:
filter (>3) [1,5,3,2,1,6,4,3,2,1]

[5,6,4]

In [39]:
-- find the largest number under 100,000 that's divisible by 3829
largestDivisible :: (Integral a) => a  
largestDivisible = head (filter p [100000,99999..])  
    where p x = x `mod` 3829 == 0  

In [41]:
-- find the sum of all odd squares that are smaller than 10,000
sum (takeWhile (<10000) (filter odd (map (^2) [1..])))

166650

In [42]:
-- same, but with list comprehension
sum (takeWhile (<10000) [n^2 | n <- [1..], odd (n^2)])  

166650

In [43]:
chain :: (Integral a) => a -> [a]  
chain 1 = [1]  
chain n  
    | even n =  n:chain (n `div` 2)  
    | odd n  =  n:chain (n*3 + 1)  

In [44]:
chain 10

[10,5,16,8,4,2,1]

In [45]:
chain 30

[30,15,46,23,70,35,106,53,160,80,40,20,10,5,16,8,4,2,1]

In [46]:
-- for all starting numbers between 1 and 100, how many chains have a length greater than 15
numLongChains :: Int  
numLongChains = length (filter isLong (map chain [1..100]))  
    where isLong xs = length xs > 15 

In [48]:
-- Applying only one parameter to a function which takes two
-- returns a function taking one parameter
let listOfFuns = map (*) [0..]  
(listOfFuns !! 4) 5 

20

## Lambdas

In [49]:
-- instead of using the 'where' clause
numLongChains :: Int  
numLongChains = length (filter (\xs -> length xs > 15) (map chain [1..100]))  

In [50]:
zipWith (\a b -> (a * 30 + 3) / b) [5,4,3,2,1] [1,2,3,4,5]  

[153.0,61.5,31.0,15.75,6.6]

## Only folds and horses

In [51]:
-- implementing 'sum' using folding instead of recursion
sum' :: (Num a) => [a] -> a  
sum' xs = foldl (\acc x -> acc + x) 0 xs  

In [52]:
sum' [3,5,2,1] 

11

In [53]:
 -- even more succintly
sum' :: (Num a) => [a] -> a  
sum' = foldl (+) 0  

In [54]:
-- checks if a value is part of a list
elem' :: (Eq a) => a -> [a] -> Bool  
elem' y ys = foldl (\acc x -> if x == y then True else acc) False ys

In [55]:
maximum' :: (Ord a) => [a] -> a  
maximum' = foldr1 (\x acc -> if x > acc then x else acc)  
  
reverse' :: [a] -> [a]  
reverse' = foldl (\acc x -> x : acc) []  
  
product' :: (Num a) => [a] -> a  
product' = foldr1 (*)  
  
filter' :: (a -> Bool) -> [a] -> [a]  
filter' p = foldr (\x acc -> if p x then x : acc else acc) []  
  
head' :: [a] -> a  
head' = foldr1 (\x _ -> x)  
  
last' :: [a] -> a  
last' = foldl1 (\_ x -> x)  

In [56]:
-- scanl is like foldl, only it reports all intermediate
-- accumulator states in the form of a list
scanl (+) 0 [3,5,2,1]  

[0,3,8,10,11]

## Function application with $

In [64]:
:t ($)

($) :: forall a b. (a -> b) -> a -> b

In [59]:
sqrt 3 + 4 + 5

10.732050807568877

In [60]:
(((sqrt 3) + 4) + 5)

Line 1: Redundant bracket
Found:
(((sqrt 3) + 4) + 5)
Why not:
((sqrt 3) + 4) + 5Line 1: Redundant bracket
Found:
(sqrt 3) + 4
Why not:
sqrt 3 + 4

10.732050807568877

In [61]:
sqrt $ 3 + 4 + 5

3.4641016151377544

In [63]:
-- we can use $ to apply functions to arguments
map ($ 3) [(4+), (10*), (^2), sqrt]  

[7.0,30.0,9.0,1.7320508075688772]

## Function composition

In [65]:
:t (.)

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

In [66]:
-- negate a list of numbers
map (\x -> negate (abs x)) [5,-3,-6,7,-3,2,-19,24]  

Line 1: Avoid lambda
Found:
\ x -> negate (abs x)
Why not:
negate . abs

[-5,-3,-6,-7,-3,-2,-19,-24]

In [68]:
-- clearer syntax with function composition
map (negate . abs) [5,-3,-6,7,-3,2,-19,24]  

[-5,-3,-6,-7,-3,-2,-19,-24]

In [69]:
oddSquareSum :: Integer  
oddSquareSum = sum . takeWhile (<10000) . filter odd . map (^2) $ [1..] 